In [27]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup

In [28]:
problems = pd.read_csv('errors_csv3.csv')

In [29]:
errors = problems['type'] == 'e'

In [30]:
problems[errors]

,id,type
1030,129117,e
1151,129351,e
1201,129439,e
1305,129635,e
1376,128663,e
1381,128669,e
1438,128769,e
1477,128843,e
1492,128871,e
1526,128933,e


In [35]:
def get_fighter_matchups(fighter_id):
    page = r.get("http://www.sherdog.com/fighter/index?id={}".format(fighter_id), timeout=3).text
    soup = BeautifulSoup(page, 'html.parser')
    found_divs = soup.findAll('div', {"class": "module fight_history"})
    for div in found_divs:
        print(div.find('h2'))
        if (div.find('h2').text == 'Fight History - Pro'):
            table = div
            break
    else:
        raise Exception('Pro fight div not found')
    fighter_name = soup.find('h1', {'itemprop': 'name'}).span.contents[0]
    fighter_table_prep = fighter_id, fighter_name
    cells = []

    event_date = 'NO FIGHTS FOUND'
    return None

    for row in table.findAll('tr')[1:]:
        result_cell, fighter_cell, event_cell, *blah = row.findAll('td')
        result = result_cell.span.contents[0]
        opponent_id_url = fighter_cell.contents[0]['href']
        opponent_id = int(opponent_id_url[opponent_id_url.rfind("-") + 1:])
        event_link = event_cell.contents[0]['href']
        event_date = event_cell.findAll('span')[-1].contents[0]
        fighter_out = (fighter_id, opponent_id, result, event_date, event_link)
        cells += [fighter_out]
    return fighter_table_prep, cells, event_date

In [24]:
x = get_fighter_matchups(129351)

<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>


In [36]:
error_id = problems[errors].head(50)['id']
[get_fighter_matchups(n) for n in error_id]

<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Pro</h2>
<h2>Upcoming Fights</h2>
<h2>Fight History - Amateur</h2>


Exception: Pro fight div not found

In [ ]:
out

In [58]:
a = ['poo']
a[-1]

'poo'

In [90]:
for x in [1,2,3]:
    if x == 2:
        print(x)
        break
else:
    print('ppoooo')

2
